# Scalar diffusion and convection-diffusion problem on channel geometry with uniform mesh using Gridap  

## Import libraries 

In [1]:
using Gridap

## Section 1:/ Diffusion Problem 
1. how to extend to non-linear diffusion? See [example](https://gridap.github.io/Tutorials/stable/pages/t004_p_laplacian/)
2. how to extend to time-dependent diffusion? See [example](https://gridap.github.io/Tutorials/dev/pages/t017_poisson_transient/). 
3. how to extend to vector-valued problems? See [example](https://gridap.github.io/Tutorials/stable/pages/t009_stokes/)
4. how to accomodate parallel computing? See [tutorial](https://gridap.github.io/Tutorials/dev/pages/t016_poisson_distributed/)

### Section 1.1:/ Linear Diffusion 

In [2]:
domain = (0,3,0,1)
partition = (300,100)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

In [3]:
# Define Lagrangian reference element
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = [7,8])
Uh = TrialFESpace(Vh, [0, 0]) # Left boundary: u = 0, right boundary: u = 0

TrialFESpace()

In [4]:
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [5]:
ff(x) = 100*exp(-100*(x[1]-1.5)^2 - 100*(x[2]-.5)^2)
sigma(x) = 1+0*((x[1]>1)*(x[1]<2)*(x[2]>.5)*(x[2]<.75)) 
a(u,v) = ∫( sigma*∇(v)⋅∇(u))*dΩ
b(v) = ∫( ff*v )*dΩ

b (generic function with 1 method)

In [6]:
op = AffineFEOperator(a,b,Uh,Vh)
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 30000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 15027091868446829417

In [7]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)

In [8]:
domain = (0,3,0,1)
partition = (300,100)
model = CartesianDiscreteModel(domain,partition)
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = [7,8])
Uh = TrialFESpace(Vh, [0, 0]) # Left boundary: u = 0, right boundary: u = 0
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
ff(x) = 100*exp(-100*(x[1]-.15)^2 - 100*(x[2]-.5)^2)
a(u,v) = ∫( ∇(v)⋅∇(u))*dΩ
b(v) = ∫( ff*v )*dΩ
op = AffineFEOperator(a,b,Uh,Vh)
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver,op)
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)

### Section 2.1:/ Linear diffusion with assembly of stiffness matrix
For explanation, see [Tutorial 13: Low Level Implementation](https://gridap.github.io/Tutorials/dev/pages/t013_poisson_dev_fe/#A-low-level-implementation-of-the-Jacobian-integration-and-assembly-1).  

In [20]:
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

#### Building argument iwq 

In [17]:
Th = Triangulation(model)
Qh = CellQuadrature(Th,4*order)

du = get_trial_fe_basis(Uh)
dv = get_fe_basis(Vh)
jac = integrate(∇(dv)⋅∇(du),Qh)

iwq = jac

30000-element Fill{Matrix{Float64}}, with entries equal to [0.6666666666666665 -0.16666666666666666 -0.16666666666666666 -0.3333333333333332; -0.16666666666666666 0.6666666666666665 -0.3333333333333332 -0.16666666666666666; -0.16666666666666666 -0.3333333333333332 0.6666666666666665 -0.16666666666666666; -0.3333333333333332 -0.16666666666666666 -0.16666666666666666 0.6666666666666664]

#### Building argument \sigma_k

In [18]:
sigmak = get_cell_dof_ids(Uh)

30000-element Table{Int32, Vector{Int32}, Vector{Int32}}:
 [1, 2, -1, 302]
 [2, 3, 302, 303]
 [3, 4, 303, 304]
 [4, 5, 304, 305]
 [5, 6, 305, 306]
 [6, 7, 306, 307]
 [7, 8, 307, 308]
 [8, 9, 308, 309]
 [9, 10, 309, 310]
 [10, 11, 310, 311]
 [11, 12, 311, 312]
 [12, 13, 312, 313]
 [13, 14, 313, 314]
 ⋮
 [29891, 29892, 30191, 30192]
 [29892, 29893, 30192, 30193]
 [29893, 29894, 30193, 30194]
 [29894, 29895, 30194, 30195]
 [29895, 29896, 30195, 30196]
 [29896, 29897, 30196, 30197]
 [29897, 29898, 30197, 30198]
 [29898, 29899, 30198, 30199]
 [29899, 29900, 30199, 30200]
 [29900, 29901, 30200, 30201]
 [29901, 29902, 30201, 30202]
 [29902, -198, 30202, 30203]

In [19]:
assem = SparseMatrixAssembler(Uh,Vh);

rs = ([iwq],[sigmak],[sigmak])

A = allocate_matrix(assem,rs)

A = assemble_matrix!(A,assem,rs)

30203×30203 SparseArrays.SparseMatrixCSC{Float64, Int64} with 269415 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦

## Section 2:/ Non-Linear Diffusion 

In [223]:
domain = (0,3,0,1)
partition = (30,10)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

In [224]:
# Define Lagrangian reference element
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = [7,8])
Uh = TrialFESpace(Vh, [0, 1]) # Left boundary: u = 0, right boundary: u = 1

TrialFESpace()

In [225]:
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [238]:
# working version 
ff(x) = 0
a(u,v) = ∫( ∇(v)⋅∇(u) + u*u*v)*dΩ
b(v) = ∫( ff*v )*dΩ

b (generic function with 1 method)

In [239]:
op = AffineFEOperator(a,b,Uh,Vh)

AffineFEOperator()

In [240]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [244]:
?LUSolver

search: LUSolver



```
struct LUSolver <: LinearSolver end
```

Wrapper of the LU solver available in julia


In [249]:
print(op)

AffineFEOperator()

In [241]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 300
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10336619661980285595

In [243]:
?solve

search: solve solve! solver NLSolver LUSolver FESolver LinearFESolver



```
solve(nls::NonlinearSolver,op::NonlinearOperator)
```

Creates and uses a zero initial guess.

---

```
solve(ls::LinearSolver,A::AbstractMatrix,b::AbstractVector)
```

---

Solve that allocates, and sets initial guess to zero and returns the solution

---



In [242]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)

## Section 3:/ Convection-Diffusion Problem 

In [215]:
domain = (0,3,0,1)
partition = (30,10)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

In [216]:
# Define Lagrangian reference element
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = [7,8])
Uh = TrialFESpace(Vh, [0, 1]) # Left boundary: u = 0, right boundary: u = 1

TrialFESpace()

In [217]:
degree = 1
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [218]:
# working version 
ff(x) = 0
velocity(x) = VectorValue( 1.0, 0.0 )
a(u,v) = ∫( 0.1*∇(v)⋅∇(u) + (velocity ⋅ ∇(u))*v)*dΩ
b(v) = ∫( v*ff )*dΩ

b (generic function with 1 method)

In [219]:
op = AffineFEOperator(a,b,Uh,Vh)

AffineFEOperator()

In [220]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [221]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 300
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 221304387296365017

In [222]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)

## Section 4: Linear Elasticity

In [34]:
domain = (0,1,0,1,0,1)
partition = (10,10,10)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

In [35]:
order = 1
refFE = ReferenceFE(lagrangian,VectorValue{3,Float64},order)

(Lagrangian(), (VectorValue{3, Float64}, 1), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [36]:
V0 = TestFESpace(model, refFE; conformity=:H1,
                dirichlet_masks=[(true, true, true), (true, true, true)])
g1(x,t::Real) = VectorValue(0.0,0.0,0.0)
g1(t::Real) = x -> g1(x,t)
U = TransientTrialFESpace(V0,[g1,g1])

TransientTrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{VectorValue{3, Int32}}}, TrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{VectorValue{3, Int32}}}}}(UnconstrainedFESpace(), [g1, g1], TrialFESpace())

In [37]:
x0 = VectorValue(0.3, 0.05, 0.3)
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [38]:
amplitude = 1000000000
deviation = 100
direction = VectorValue(0,0,1)
f_t(x) = amplitude*exp(-deviation*norm(x-x0))*direction
f = interpolate_everywhere(f_t, V0)

m(utt,v) = ∫(v⊙utt)dΩ
c(ut,v) = ∫(v⊙ut)dΩ
a(u,v) = ∫(ε(v) ⊙ (ε(u)))dΩ 
b(t,v) = ∫(v⋅f_t)dΩ
m(t,utt,v) = m(utt,v)
c(t,ut,v) = c(ut,v)
a(t,u,v) = a(u,v)
res(t,u,v) = m(∂tt(u),v) + c(∂t(u),v) + a(u,v) - b(t,v)
jac(t,u,du,v) = a(du,v)
jac_t(t,u,dut,v) = c(dut,v)
jac_tt(t,u,dutt,v) = m(dutt,v)
op = TransientFEOperator(res, U, V0; order=2) # auto differentiation

TransientFEOperatorFromWeakForm()

In [39]:
linear_solver = LUSolver()
dt = 0.1
γ = 0.5
β = 0.25
u₀ = interpolate_everywhere(VectorValue(0,0,0),U(0))
v₀ = interpolate_everywhere(VectorValue(0,0,0), U(0))
a₀ = interpolate_everywhere(VectorValue(0,0,0), U(0))

ode_solver = Newmark(linear_solver, dt, γ, β)

t₀ = 0.0
T = 10.0
uht = solve(ode_solver,op,(u₀,v₀, a₀),t₀,T)

Gridap.ODEs.TransientFETools.TransientFESolution(GenericODESolution(), TransientTrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{VectorValue{3, Int32}}}, TrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{VectorValue{3, Int32}}}}}(UnconstrainedFESpace(), [g1, g1], TrialFESpace()))

In [42]:
createpvd("elasticity_transient_solution") do pvd
  for (uₕ,t) in uht
    pvd[t] = createvtk(Ω,"elasticity_transient_solution_$t"*".vtu",cellfields=["u"=>uₕ])
  end
end

101-element Vector{String}:
 "elasticity_transient_solution.pvd"
 "elasticity_transient_solution_0.1.vtu"
 "elasticity_transient_solution_0.2.vtu"
 "elasticity_transient_solution_0.30000000000000004.vtu"
 "elasticity_transient_solution_0.4.vtu"
 "elasticity_transient_solution_0.5.vtu"
 "elasticity_transient_solution_0.6.vtu"
 "elasticity_transient_solution_0.7.vtu"
 "elasticity_transient_solution_0.7999999999999999.vtu"
 "elasticity_transient_solution_0.8999999999999999.vtu"
 "elasticity_transient_solution_0.9999999999999999.vtu"
 "elasticity_transient_solution_1.0999999999999999.vtu"
 "elasticity_transient_solution_1.2.vtu"
 ⋮
 "elasticity_transient_solution_8.899999999999984.vtu"
 "elasticity_transient_solution_8.999999999999984.vtu"
 "elasticity_transient_solution_9.099999999999984.vtu"
 "elasticity_transient_solution_9.199999999999983.vtu"
 "elasticity_transient_solution_9.299999999999983.vtu"
 "elasticity_transient_solution_9.399999999999983.vtu"
 "elasticity_transient_solution_9.